## Practical basics

It is easy to calculate the quantities we encounter most often in statistics!

# Very WIP!

Basic statistics - practical skills
- Maybe start with counting and summing?
- For the card experiment, how to calculate...
    - mean #wins, standard deviation, standard error
    - sample size vs error?
- More general, how to characterise a sample (distribution)
    - mean, median, mode
    - standard deviation, standard error
    - quantiles and box plots
- How to calculate them with pen & pape for small examples?!?
- Appendix: how to calculate the standard quantities in Excel, Tableau, Python

How to calculate these quantities
- mean
- standard deviation
- standard error

[no, not this mean](https://en.wikipedia.org/wiki/File:Taylor_Swift_-_Mean.png#/media/File:Taylor_Swift_-_Mean.png)

if you fount this funny, you may like [this list.](https://www.isg.rhul.ac.uk/~sdg/mathsongs.html) You may also have issues.

In [1]:
import altair as alt
import helpers.plotting as pt
pt.enable_slide_theme()
pt.import_lato_font_in_notebook()

In [2]:
# Simple example for statistical measures:
# drawing coins from a pocket

In [3]:
import pandas as pd
import numpy as np

In [4]:
from helpers.svg_wrapper import SVGImg

In [141]:
from scipy.stats import mode

In [126]:
# test: simple example to calculate mentally

#coins_in_pocket = [1,2,2,5,20]
#coins_in_pocket = [1,1,1,2,5,5,20] # we go with this one
#coins_in_pocket = [1,1,2,2,2,5,5,10,20]
#coins_in_pocket = [1,1,1,2,2,5,10,20]
#coins_in_pocket = [5,5,10,20,50]
coins_in_pocket = [5, 10, 10, 20, 20]
#coins_in_pocket = [ 2, 10, 10, 10, 50, 50]

print("Sum of coins:", sum(coins_in_pocket))

observations = pd.DataFrame(
    coins_in_pocket,
    columns = ['value'],
    index = pd.Index(range(1,len(coins_in_pocket)+1), name='draw')
)

observations.describe()

Sum of coins: 65


,value
count,5.000000
mean,13.000000
std,6.708204
min,5.000000
25%,10.000000
50%,10.000000
75%,20.000000
max,20.000000


In [150]:
np.sum(observations.values**2), np.mean(observations.values)**2

(1025, 169.0)

In [151]:
np.mean(observations.values**2),  np.mean(observations)**2

(205.0,
 value    169.0
 dtype: float64)

In [152]:
np.var(observations.values), np.std(observations.values)#*len(observations)

(36.0, 6.0)

In [168]:
[(c - np.mean(coins_in_pocket))**2 for c in coins_in_pocket]

[64.0, 9.0, 9.0, 49.0, 49.0]

In [153]:
np.mean((np.array(coins_in_pocket) - np.mean(coins_in_pocket))**2 )

36.0

In [154]:
((observations - observations.mean())**2).mean()

value    36.0
dtype: float64

In [155]:
np.var(observations), observations.var()

(value    36.0
 dtype: float64,
 value    45.0
 dtype: float64)

In [170]:
np.sqrt(len(observations))

2.23606797749979

In [180]:
np.var(coins_in_pocket)/ len(coins_in_pocket)

7.2

In [185]:
np.var(coins_in_pocket, ddof=1)

6.708203932499369

In [181]:
observations.var() / len(observations)

value    9.0
dtype: float64

In [158]:
print('Standard error:', float(observations.std() / np.sqrt(len(observations))))

Standard error: 3.0


In [203]:
np.quantile(coins_in_pocket, .75)

20.0

## How to characterise a sample of quantitative observations?

- **Central tendency**: where are the observed values?
- **Dispersion**: how far apart are the observed values?

### For example, imagine drawing coins from a pocket

- How much is each new coin worth?
- How certain are we about that value?

Let's start with an example of 5 coins...

### Measures of central tendency

In [159]:
SVGImg('images/coins_central_tendency.svg', width='70%', output_dir='slides')

## *

If there is an even number of observations in our sample, <br>the median is in between the two central ones:
- median([5,5,**10**,20,20]) = **10**
- median([5,5,**10**,**20**,20,20]) = (**10**+**20**) / 2 = 15

See e.g. https://en.wikipedia.org/wiki/Central_tendency for details <br>on these and other measures of central tendency.

### Measures of dispersion

In [205]:
SVGImg('images/coins_dispersion.svg', width='70%', output_dir='slides')

See e.g. https://en.wikipedia.org/wiki/Statistical_dispersion for details <br>on these and other measures of dispersion.

### Try it!

In [230]:
# test: interactive distribution parameters
plot_width  = 500
max_samples = 200
input_mean = alt.binding(
    input='range',
    min=-5,
    max=5,
    step=.1,
    name='mean'
)
input_std = alt.binding(
    input='range',
    min=0.1,
    max=5,
    step=.1,
    name='std'
)
input_samples = alt.binding(
    input='range',
    min=10,
    max=max_samples,
    step=1,
    name='samples'
)
mean_selection = alt.selection_single(
    bind=input_mean,
    init={'mean': 0}
)
std_selection = alt.selection_single(
    bind=input_std,
    init={'std': 1}
)
samples_selection = alt.selection_single(
    bind=input_samples,
    init={'samples': 50}
)
scales = alt.selection_interval(bind='scales')
                                
data = alt.sequence(0, max_samples, as_='t')
chart = alt.Chart(data).add_selection(
    scales, samples_selection, std_selection, mean_selection
).transform_filter(
    alt.datum.t <= alt.expr.toNumber(samples_selection.samples)    
).transform_calculate(
  mean = mean_selection['mean'],
  std  = std_selection['std'],
).transform_calculate(
    t='datum.t',
    x='sampleNormal(+datum.mean,+datum.std)', # the plus casts to number
)
scale = alt.Scale(domain=[-10,10])
hist = chart.mark_bar().encode(
    alt.X("x:Q", bin=alt.Bin(maxbins=7), scale=scale, axis=None),
    y='count()',
).properties(
    width=plot_width
)
ticks = chart.mark_tick().encode(
    x=alt.X('x:Q', scale=scale)
).properties(
    view=alt.ViewConfig(strokeWidth=0), width=plot_width)

(hist & ticks)

alt.VConcatChart(...)

Each time you change the parameters, new samples are drawn from a normal distribution.<br>
The latter is also called Gaussian or bell curve and we will discuss it depth later.

In [9]:
import numpy as np
#from helpers.cards import draw_cards